# Créer un fichier d’index

Le fichier d’index est destiné à recueillir les métadonnées sur les épisodes de la série.

Pour le moment, ces métadonnées seront constituées du :
- titre de l’épisode
- numéro d’ordre de la saison
- numéro d’ordre de l’épisode dans la saison
- nom du transcripteur
- source de la transcription (Hypnoweb)

Elles seront suivies plus tard par d’autres, comme la liste des locuteurs, le nombre de répliques, le nombres de mots…

Idéalement, ce fichier d’index se conçoit sous la forme d’un fichier à plat, contenant une ligne par épisode, chaque métadonnée séparée de la précédente par un signe.

## Récupérer les métadonnées de l’épisode

À partir du nom même d’un script, il est possible de récupérer trois informations : le titre de l’épisode, son numéro d’ordre dans la saison et le numéro d’ordre de la saison à laquelle il est rattaché.

**Format d’un nom de fichier :** `S00E00-titre-episode.txt`  
**Exemple :** `S01E02-la-tarte-aux-myrtilles.txt`

Grâce aux expressions rationnelles, il est aisé de capturer ces métadonnées et de les inclure dans le fichier d’index.

Par exemple, si pour le fichier `S01E02-la-tarte-aux-myrtilles.txt` on souhaite récupérer le numéro d’ordre de la saison (composé de deux chiffres après le caractère `S`) :

In [ ]:
import re

filename = 'S01E02-la-tarte-aux-myrtilles.txt'
pattern = 'S(?P<season>[0-9]{2})'
m = re.match(pattern, filename)
print(m.group('season'))

À présent, il ne reste plus qu’à appliquer la méthode aux autres métadonnées et sur l’ensemble des fichiers :

In [ ]:
# Modules to import
import os, re

# Separator
sep = ','

# Pattern to capture the number of the season and the number of the episode in the season
# and the title of the episode.
pattern = 'S(?P<season>[0-9]{2})E(?P<episode>[0-9]{2,3})-(?P<title>.+).txt'

# Empty string
output = ''

# Opens the scripts and captures the metadata
for screenplay in os.listdir('./sample'):
    with open(f'./sample/{screenplay}') as file:
        metas = re.match(pattern, screenplay)
        output += f"{metas.group('title')}{sep}"
        output += f"{metas.group('season')}{sep}"
        output += f"{metas.group('episode')}\n"

## Récupérer le pseudonyme du transcripteur

Pour rappel, le pseudonyme du transcripteur est reporté à la dernière ligne d’un script, mais malheureusement pas de manière systématique.

La solution consiste à tester l’écriture de cette ligne et, si elle débute par la chaîne *Rédigé par*, alors capturer le pseudonyme et l’ajouter à `output`.

Exemple avec deux lignes issues du corpus :

In [ ]:
import re

output = 'ambidextrie,01,35'

pattern = 'Rédigé par\s?(?P<alias>[A-Za-z1-9]+)\s?.+'
last_lines = ['Rédigé par Holly95 pour HypnoseriesKaamelott', '…']

for last_line in last_lines:
    if last_line.startswith('Rédigé par'):
        m = re.match(pattern, last_line)
        output += sep + m.group('alias')
        print(output)

Intégrer la méthode à l’ensemble des scripts :

In [ ]:
import os, re

sep = ','
pattern = 'S(?P<season>[0-9]{2})E(?P<episode>[0-9]{2,3})-(?P<title>.+).txt'
output = ''

for screenplay in os.listdir('./sample'):
    with open(f'./sample/{screenplay}') as file:
        metas = re.match(pattern, screenplay)
        output += f"{metas.group('title')}{sep}"
        output += f"{metas.group('season')}{sep}"
        output += f"{metas.group('episode')}{sep}"
        # Focus on the last line
        last_line = file.readlines()[-1]
        if last_line.startswith('Rédigé par'):
            m = re.match('Rédigé par\s?(?P<alias>[A-Za-z1-9]+)\s?.+', last_line)
            output += f"{m.group('alias')}\n"

## Renseigner la source

La dernière métadonnée concerne la source de la transcription, à savoir le site Hypnoweb. Elle se rajoute manuellement :

In [ ]:
output = 'ambidextrie,01,35,Holly95'
output += sep + 'Hypnoweb'
print(output)

Pour l’ensemble des scripts :

In [ ]:
import os, re

sep = ','
pattern = 'S(?P<season>[0-9]{2})E(?P<episode>[0-9]{2,3})-(?P<title>.+).txt'
output = ''

for screenplay in os.listdir('./sample'):
    with open(f'./sample/{screenplay}') as file:
        metas = re.match(pattern, screenplay)
        output += f"{metas.group('title')}{sep}"
        output += f"{metas.group('season')}{sep}"
        output += f"{metas.group('episode')}{sep}"
        last_line = file.readlines()[-1]
        if last_line.startswith('Rédigé par'):
            m = re.match('Rédigé par\s?(?P<alias>[A-Za-z1-9]+)\s?.+', last_line)
            output += f"{m.group('alias')}"
        
        output += f'{sep}Hypnoweb\n'

## Enregistrer le résultat

Finalement, enregistrer la sortie dans un fichier :

In [ ]:
with open('./files/index.txt', 'w') as index:
    index.write(output)